In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


"""
currently using identical architecture to the image denoising ConvNet in Sebastian's paper
"""
class SebastianConvNet(nn.Module):
    def __init__(self, in_channels, in_height, in_width): #for greyscale set channels = 1
        if in_height % 16 or in_width % 16:
            print('Error: height and width of image incompatible with architecture, must be divisible by 16')
            return
        super().__init__()
        
        self.in_channels = in_channels
        self.in_height = in_height
        self.in_width = in_width
        
        #input is size [batchsize, in_channels, in_height, in_width]
        self.conv1 = nn.Conv2d(in_channels, 16, 5, padding = 2) #[batchsize, 16, heigh, width]
        self.conv2 = nn.Conv2d(16, 32, 5, padding = 2) #[batchsize, 32, in_height, in_width]
        self.conv3 = nn.Conv2d(32, 32, 5, stride = 2, padding = 2) #batchsize, 32, in_height / 2, in_width / 2]
        self.conv4 = nn.Conv2d(32, 64, 5, stride = 2, padding = 2) #[batchsize, 64, in_height / 4, in_width / 4]
        self.conv5 = nn.Conv2d(64, 64, 5, stride = 2, padding = 2) #batchsize, 64, in_height / 8, in_width / 8]
        self.conv6 = nn.Conv2d(64, 128, 5, stride = 2, padding = 2) #[batchsize, 128, in_height / 16, in_width / 16]
        
        remaining_pixels = in_height * in_width // (16 * 16)
        self.remaining_dimensions = remaining_pixels * 128
        
        #after reshaping, the input to the feedforward layers will be [batchsize, remaining_dimension]
        self.linear1 = nn.Linear(self.remaining_dimensions, 256) #[batchsize, 256]
        self.linear2 = nn.Linear(256, 1) #[batchsize, 1]
    
    def forward(self, batch):
        #batch must be a torch.tensor on device = device, of dtype = torch.float, and of size [any, self.in_channels, self.in_height, self.in_width]
        if list(batch.size())[1:] != [self.in_channels, self.in_height, self.in_width]:
            print('Error: channels, height, and width different to initialisation')
            return
        
        layer1 = F.leaky_relu(self.conv1(batch), negative_slope = 0.1)
        layer2 = F.leaky_relu(self.conv2(layer1), negative_slope = 0.1)
        layer3 = F.leaky_relu(self.conv3(layer2), negative_slope = 0.1)
        layer4 = F.leaky_relu(self.conv4(layer3), negative_slope = 0.1)
        layer5 = F.leaky_relu(self.conv5(layer4), negative_slope = 0.1)
        layer6 = F.leaky_relu(self.conv6(layer5), negative_slope = 0.1)
        layer6_reshape = layer6.view(-1, self.remaining_dimensions)
        layer7 = F.leaky_relu(self.linear1(layer6_reshape), negative_slope = 0.1)
        output = self.linear2(layer7)
        return output #[batchsize, 1]

In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

"""
for now we assume we generate all training data beforehand as numpy arrays. we then convert to pytorch tensors and store on the cpu memory, converting to gpu memory when needed
"""
"""
if memory or speed becomes an issue we could rewrite the training data generation on the gpu using pytorch's linear algebra, and generate it in situ while training
"""

def train(NN, groundtruth_numpy, chanvese_numpy, epochs = 7, batch_size = 100, mu = 20, lr = 0.0001, device = "cuda:0" if torch.cuda.is_available() else "cpu"):
    """
    NN is the neural network, e.g. could initialise by NN = SebastianConvNet(1, 256, 256)
    """
    """
    groundtruth_numpy is numpy array of [batchsize, image_channels, image_height, image_width] groundtruth segmentations
    chanvese_numpy is numpy array of [batchsize, image_channels, image_height, image_width] chanvese segmentations
    the two datasets do not have to contain corresponding images for the purpose of training (see paper for why) INFACT THEY SHOULDN'T (should probably incorporate this by shuffling beforehand, or could potentially include a shuffle command here)
    """
    
    NN.to(device)
    optimiser = optim.RMSprop(NN.parameters(), lr = lr) #not sure why Sebastian doesn't use Adam, but hey
    
    #object to allow easy access to batches of training data
    dataset = DataLoader(TensorDataset(torch.from_numpy(groundtruth_numpy).float(), torch.from_numpy(chanvese_numpy).float()), batch_size = batch_size, pin_memory = True, drop_last = True)
    
    for i in range(epochs):
        """
        haven't got a log keeping track of training progress at the moment
        """
        for groundtruth_batch, chanvese_batch in dataset:
            """
            don't currently do any shuffling of the dataset, just pass through the entire dataset (in batches), once per epoch
            not sure what Sebastian does
            """
            assert groundtruth_batch.size() == chanvese_batch.size()
            
            groundtruth_batch = groundtruth_batch.to(device)
            chanvese_batch = chanvese_batch.to(device)
            
            batchsize = groundtruth_batch.size(0)
            
            #intermediate point
            epsilon = torch.rand([batchsize], device = device) #[batchsize]
            epsilon = epsilon.unsqueeze(1).unsqueeze(2).unsqueeze(3).expand(groundtruth_batch.size()) #[batchsize, channels, height, width]
            intermediate_batch = epsilon * groundtruth_batch + (1 - epsilon) * chanvese_batch #[batchsize, channels, height, width]
            intermediate_batch.requires_grad = True
            
            #apply the neural network
            groundtruth_NN = NN(groundtruth_batch) #[batchsize]
            chanvese_NN = NN(chanvese_batch) #[batchsize]
            intermediate_NN = NN(intermediate_batch) #[batchsize]
            intermediate_NN = intermediate_NN.sum() #[1] trick to compute all gradients in one go
            
            #calculate the loss
            wasserstein_loss = (groundtruth_NN - chanvese_NN).mean() #[1]
            gradient = torch.autograd.grad(intermediate_NN, intermediate_batch, create_graph = True)[0] #[batchsize, channels, height, width], must create_graph so can backprop again (second derivatives) during gradient descent
            gradient_loss = (F.relu(gradient.square().sum((1, 2, 3)).sqrt())).square().mean() #[1]
            loss = wasserstein_loss + mu * gradient_loss #[1]
            
            #backprop step
            optimiser.zero_grad() #no need to zero the gradients of the intermediate point, since it is reinitialised each batch
            loss.backward()
            optimiser.step()
    
    return NN.to("cpu")

In [64]:
import numpy as np
import torch
import torch.nn.functional as F

"""
the regularisation parameter lambda CANNOT be initialised in the same way for segmentation as for denoising
"""
"""
the function unreg_mini in Sebastian's paper shouldn't need to be redone here, as chanvese should already be a good starting point for reconstruction
"""


"""
must recalculate data fitting term in pytorch, so we can compute gradients
"""
"""
I THINK THIS IS THE IMPLEMENTATION MIKE WANTED
"""
def data_fitting(chanvese_batch, noisy_batch, lambda_chanvese = 1, threshold = 0.5, c1 = None, c2 = None, alpha = None):
    """
    chanvese_batch & noisy_batch must a torch.tensor (ideally on gpu) of size [batchsize, 1, height, width]
    noisy_batch contains the corresponding noisy images to chanvese_batch
    threshold is used to find the segmentation boundary (for the purpose of calculating c1, c2) from chanvese_batch
    """
    assert chanvese_batch.size() == noisy_batch.size()
    assert chanvese_batch.size(1) == 1 #require greyscale image, i.e. only one channel
    
    #calculate c1, c2 implicity from u
    """
    I DO THINK we want to backprop along c1, c2 when performing the reconstruction (algorithm 2), only relevant when c1, c2 are calculated implicitly
    """
    if c1 == None:
        c1 = (noisy_batch * (chanvese_batch > threshold)).mean((1, 2, 3)) #[batchsize]
    if c2 == None:
        c2 = (noisy_batch * (chanvese_batch <= threshold)).mean((1, 2, 3)) #[batchsize]
    
    chanvese_term = lambda_chanvese * ((noisy_batch - c1.unsqueeze(1).unsqueeze(2).unsqueeze(3)).square() - (noisy_batch - c2.unsqueeze(1).unsqueeze(2).unsqueeze(3)).square()) #[batchsize, 1, height, width]
    
    #calculate alpha implicity from u, lambda, c1, and c2?
    """
    I DON'T THINK we want to backprop along alpha when performing the reconstruction (algorithm 2), only relevant when alpha is calculated implicitly
    hence .detach() below
    """
    if alpha == None:
        alpha = chanvese_term.detach().abs().max(3)[0].max(2)[0].max(1)[0] #[batchsize]
    
    penality_term = 2 * ((chanvese_batch - 0.5).abs() - 1) #[batchsize, 1, height, width]
    penality_term = penality_term * (penality_term > 0) #[batchsize, 1, height, width]
    
    """
    integral over domain is just done by taking the mean, should just correspond to scaling lambda_reg accordingly in reconstruct (below)
    """
    datafitting_term = (chanvese_term * chanvese_batch + alpha.unsqueeze(1).unsqueeze(2).unsqueeze(3) * penality_term).mean((1, 2, 3)) #[batchsize]
    
    return datafitting_term #[batchsize]


"""
ALGORITHM 2:
simultaneously perform a number of gradient descent steps on a full batch of chanvese segmentations, or already partialy reconstructed segmentations
(both would take the argument chanvese_batch below)
noisy_batch contains the corresponding noisy images to chanvese_batch (for the purpose of the datafitting term above)
"""
def reconstruct(chanvese_batch, noisy_batch, NN, lambda_reg, epsilon, reconstruction_steps = 1):
    """
    chanvese_batch & noisy_batch must be a torch.tensor of size [batchsize, channels, height, width]
    NN is the learnt regulariser
    lambda_reg is how much we weight the regularising term (not the datafitting term) when reconstructing the solution according to algorithm 2
    """
    device = next(NN.parameters()).device #trick to find device NN is stored on
    reconstructed_batch = chanvese_batch.to(device).detach() #transfer chanvese_batch to same device NN is stored on, detach just incase
    noisy_batch_copy = noisy_batch.to(device) #transfer noisy_batch to same device NN is stored on
    
    for i in range(reconstruction_steps):
        reconstructed_batch.requires_grad = True #set requires_grad to True, gradients are initialised at zero, and entire backprop graph will be recreated (not the most efficient way, as autograd graph has to be recreated each time)
        
        """
        data_fitting function not yet implemented
        """
        datafitting = data_fitting(reconstructed_batch, noisy_batch_copy) #[batchsize]
        regularising = NN(reconstructed_batch) #[batchsize]
        
        error = datafitting + lambda_reg * regularising #[batchsize]
        error = error.sum() #[1], trick to compute all gradients in one go
        
        gradients = torch.autograd.grad(error, reconstructed_batch)[0]
        reconstructed_batch = (reconstructed_batch - epsilon * gradients).detach() #detaching from previous autograd which also sets requires_grad to False
    
    return reconstructed_batch.to(chanvese_batch.device), noisy_batch #send back to original device


"""
a quick function for evaluating the quality of the reconstructed segmentation according to the L2 difference between it and groundtruth
"""
def quality(reconstructed_batch, groundtruth_batch):
    """
    reconstructed_batch, and groundtruth_batch must be torch.tensors of the same size [batchsize, channels, height, width]
    """
    return (reconstructed_batch - groundtruth_batch).square().sum((1, 2, 3)).sqrt() #[batchsize]


"""
analogue of Sebastian's function log_minimum (except without storing any data in logs), which keeps reconstructing solutions until their quality (as defined above, i.e. requiring knowledge of the ground truth) no longer keeps decreasing

idea is to use this to evaluate performance of NN
"""
def minimum(chanvese_batch, noisy_batch, groundtruth_batch, NN, lmb, epsilon):
    
    assert chanvese_batch.size() == noisy_batch.size()
    assert chanvese_batch.size() == groundtruth_batch.size()
    assert chanvese_batch.device == noisy_batch.device
    assert chanvese_batch.device == groundtruth_batch.device
    batchsize = chanvese_batch.size(0)
    device = chanvese_batch.device
    
    todo_mask = torch.ones([batchsize], dtype = torch.bool, device = device)
    chanvese_todo = chanvese_batch
    noisy_todo = noisy_batch
    groundtruth_todo = groundtruth_batch
    quality_prev = torch.full([batchsize], float('inf'), device = device)
    minimum_batch = torch.empty_like(chanvese_todo)
    final_quality = torch.empty_like(quality_prev)
    steps = torch.zeros_like(quality_prev)
    
    while todo_mask.sum():
        steps += todo_mask
        
        chanvese_todo = reconstruct(chanvese_todo, noisy_todo, NN, lmb, epsilon)[0]
        quality_new = quality(chanvese_todo, groundtruth_todo)
        done_mask = quality_new > quality_prev
        
        print(done_mask)
        
        done_mask_unravel = torch.zeros_like(todo_mask).masked_scatter(todo_mask, done_mask)
        
        
        minimum_batch.masked_scatter_(done_mask_unravel.unsqueeze(1).unsqueeze(2).unsqueeze(3).expand(minimum_batch.size()), chanvese_todo.masked_select(done_mask.unsqueeze(1).unsqueeze(2).unsqueeze(3).expand(chanvese_todo.size())))
        
        print(minimum_batch.size())
        
        final_quality.masked_scatter_(done_mask_unravel, quality_new.masked_select(done_mask))
        todo_mask.masked_fill_(done_mask_unravel, False)
        
        chanvese_todo = chanvese_todo.masked_select(done_mask.logical_not().unsqueeze(1).unsqueeze(2).unsqueeze(3).expand(chanvese_todo.size())).view(-1, chanvese_batch.size(1), chanvese_batch.size(2), chanvese_batch.size(3))
        noisy_todo = noisy_todo.masked_select(done_mask.logical_not().unsqueeze(1).unsqueeze(2).unsqueeze(3).expand(noisy_todo.size())).view(-1, chanvese_batch.size(1), chanvese_batch.size(2), chanvese_batch.size(3))
        groundtruth_todo = groundtruth_todo.masked_select(done_mask.logical_not().unsqueeze(1).unsqueeze(2).unsqueeze(3).expand(groundtruth_todo.size())).view(-1, chanvese_batch.size(1), chanvese_batch.size(2), chanvese_batch.size(3))
        quality_prev =  quality_new.masked_select(done_mask.logical_not())
    
    return minimum_batch, final_quality, steps #outputs the final (optimal) reconstruction, their corresponding quality, and the reconstruction steps required

In [6]:
NN = SebastianConvNet(1, 16, 16)

In [9]:
training_data_chanvese = np.random.rand(100, 1, 16, 16)
training_data_groundtruth = np.random.rand(100, 1, 16, 16)
NN = train(NN, training_data_groundtruth, training_data_chanvese)

In [61]:
test_data_noisy = torch.rand([10, 1, 16, 16])
test_data_chanvese = torch.rand([10, 1, 16, 16])
test_data_groundtruth = torch.rand([10, 1, 16, 16])
reconstructed = reconstruct(test_data_chanvese, test_data_noisy, NN, 5, 0.01, 10)

In [18]:
reconstructed

(tensor([[[[0.8684, 0.8561, 0.2101,  ..., 0.3885, 0.4646, 0.4728],
           [0.0199, 0.4695, 0.5001,  ..., 0.7108, 0.4392, 0.0088],
           [0.3581, 0.7746, 0.9867,  ..., 0.9543, 0.5236, 0.3853],
           ...,
           [0.3883, 0.9577, 0.5457,  ..., 0.5605, 0.4997, 0.8651],
           [0.1854, 0.6110, 0.1350,  ..., 0.6948, 0.5146, 0.5793],
           [0.8155, 0.1323, 0.1980,  ..., 0.9252, 0.4547, 0.5242]]],
 
 
         [[[0.2000, 0.2321, 0.3727,  ..., 0.7773, 0.3836, 0.0328],
           [0.3644, 0.0545, 0.2516,  ..., 0.0100, 0.3447, 0.9356],
           [0.5361, 0.5852, 0.5147,  ..., 0.5571, 0.1468, 0.2360],
           ...,
           [0.6393, 0.5887, 0.6444,  ..., 0.8459, 0.7009, 0.8063],
           [0.2829, 0.9299, 0.4077,  ..., 0.1174, 0.1284, 0.3718],
           [0.0171, 0.0057, 0.6275,  ..., 0.8928, 0.4098, 0.1889]]],
 
 
         [[[0.2813, 0.4175, 0.6580,  ..., 0.0769, 0.1924, 0.2931],
           [0.0389, 0.3832, 0.7358,  ..., 0.2459, 0.7911, 0.6411],
           [0.9120

In [65]:
(minimum_batch, quality, steps) = minimum(test_data_chanvese, test_data_noisy, test_data_groundtruth, NN, 5, 0.01)

tensor([False, False, False, False, False, False, False, False, False, False])
torch.Size([10, 1, 16, 16])
tensor([ True,  True,  True, False,  True, False, False, False,  True,  True])
torch.Size([10, 1, 16, 16])
tensor([False, False, False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False, False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False, False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False, False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False, False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False, False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False, False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False, False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False, False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False, False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False, False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False, False, False])
torch.Size([10, 1, 16, 16])
tensor

torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
tensor([False, False])
torch.Size([10, 1, 16, 16])
ten

torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tens

torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tens

torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tensor([False])
torch.Size([10, 1, 16, 16])
tens

In [66]:
minimum_batch

tensor([[[[ 9.9347e-01,  7.7764e-02,  1.4307e-01,  ...,  8.0322e-01,
            3.8434e-01,  1.2106e-01],
          [ 9.4117e-01,  4.1654e-01,  2.3220e-01,  ...,  3.8847e-01,
            9.7980e-01,  1.0580e-01],
          [ 6.7723e-01,  6.2999e-01,  7.3034e-01,  ...,  3.9768e-02,
            2.9735e-01,  7.8122e-01],
          ...,
          [ 1.8811e-01,  3.7758e-01,  7.7706e-01,  ...,  7.6117e-01,
            5.8667e-01,  1.4761e-01],
          [ 8.5412e-01,  6.6280e-01,  4.3907e-01,  ...,  6.1349e-01,
            8.2566e-02,  3.7526e-01],
          [ 4.0901e-01,  7.1494e-01,  2.8211e-01,  ...,  5.2027e-02,
            4.6962e-01,  2.2878e-01]]],


        [[[ 3.5123e-01,  7.1520e-01,  1.6929e-01,  ...,  9.6617e-01,
            2.7461e-01,  3.2525e-01],
          [ 4.4444e-02,  1.0503e-01,  6.8730e-01,  ...,  7.3356e-01,
            5.9448e-01,  9.8135e-01],
          [ 1.7804e-01,  4.2946e-01,  3.4621e-02,  ...,  1.0273e-02,
            8.5573e-01,  3.4254e-01],
          ...,
   

In [67]:
quality

tensor([6.4136, 6.8043, 6.7174, 6.5290, 6.4447, 6.4152, 6.5873, 6.8286, 6.6949,
        6.8076])

In [68]:
steps

tensor([  2.,   2.,   2.,  94.,   2., 283., 859., 112.,   2.,   2.])

In [6]:
!black algorithm2.py

reformatted algorithm2.py
All done! \u2728 \U0001f370 \u2728
1 file reformatted.


In [7]:
!black algorithm2.py

All done! \u2728 \U0001f370 \u2728
1 file left unchanged.


In [8]:
!black classfiles

reformatted C:\Users\huysm\Documents\GitHub\Segmentation\ClassFiles\networks.py
reformatted C:\Users\huysm\Documents\GitHub\Segmentation\ClassFiles\ShapeGenerator.py
All done! \u2728 \U0001f370 \u2728
2 files reformatted.
